In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [2]:
#Deploy the Data Set
db = pd.read_csv("ecommerce_clickstream_transactions.csv")
db.head()

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,NaN
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,NaN
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,NaN
3,1,1,2024-03-12 00:32:05.495638,add_to_cart,prod_4112,NaN,NaN
4,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,NaN


In [3]:
#Fill the NaN in the outcome collumn with No purchase, as the column needs to contains either purchase or no purchase
db['Outcome'] = db['Outcome'].fillna('No purchase')

In [4]:
db.head(100)

,UserID,SessionID,Timestamp,EventType,ProductID,Amount,Outcome
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,NaN,No purchase
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,NaN,No purchase
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,NaN,No purchase
3,1,1,2024-03-12 00:32:05.495638,add_to_cart,prod_4112,NaN,No purchase
4,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,NaN,No purchase
...,...,...,...,...,...,...,...
95,2,2,2024-04-16 05:04:41.305679,purchase,prod_9382,130.052528,purchase
96,2,2,2024-04-10 21:19:06.834986,page_view,NaN,NaN,No purchase
97,2,3,2024-01-03 01:39:14.214806,click,NaN,NaN,No purchase
98,2,3,2024-03-26 17:29:34.941192,logout,NaN,NaN,No purchase


In [5]:
# Drop the 'Amount' column
db = db.drop(columns=['Amount'])


In [6]:
db

,UserID,SessionID,Timestamp,EventType,ProductID,Outcome
0,1,1,2024-07-07 18:00:26.959902,page_view,NaN,No purchase
1,1,1,2024-03-05 22:01:00.072000,page_view,NaN,No purchase
2,1,1,2024-03-23 22:08:10.568453,product_view,prod_8199,No purchase
3,1,1,2024-03-12 00:32:05.495638,add_to_cart,prod_4112,No purchase
4,1,1,2024-02-25 22:43:01.318876,add_to_cart,prod_3354,No purchase
...,...,...,...,...,...,...
74812,1000,10,2024-05-11 22:48:45.500117,purchase,prod_1238,purchase
74813,1000,10,2024-03-29 04:09:32.514318,logout,NaN,No purchase
74814,1000,10,2024-02-09 02:58:56.128697,login,NaN,No purchase
74815,1000,10,2024-04-30 16:19:48.002633,purchase,prod_2515,purchase


In [9]:
# 3. Convert Categorical Data
# Encode categorical features using Label Encoding or One-Hot Encoding
# Here we use Label Encoding as an example
label_encoder = LabelEncoder()
db['EventType'] = label_encoder.fit_transform(db['EventType'])

In [11]:
# 3. Convert Categorical Data
# Encode categorical features using Label Encoding or One-Hot Encoding
# Here we use Label Encoding as an example
label_encoder = LabelEncoder()
db['Outcome'] = label_encoder.fit_transform(db['Outcome'])

In [12]:
db

,UserID,SessionID,Timestamp,EventType,ProductID,Outcome
0,1,1,2024-07-07 18:00:26.959902,4,NaN,0
1,1,1,2024-03-05 22:01:00.072000,4,NaN,0
2,1,1,2024-03-23 22:08:10.568453,5,prod_8199,0
3,1,1,2024-03-12 00:32:05.495638,0,prod_4112,0
4,1,1,2024-02-25 22:43:01.318876,0,prod_3354,0
...,...,...,...,...,...,...
74812,1000,10,2024-05-11 22:48:45.500117,6,prod_1238,1
74813,1000,10,2024-03-29 04:09:32.514318,3,NaN,0
74814,1000,10,2024-02-09 02:58:56.128697,2,NaN,0
74815,1000,10,2024-04-30 16:19:48.002633,6,prod_2515,1


In [16]:
#Ground ProductID using SessionID to check if there is any scope of using uniuqe id to fill the missing value
features = db.groupby('SessionID').agg({
    'ProductID': 'nunique' # Number of unique products viewed
}).reset_index()

In [17]:
features

,SessionID,ProductID
0,1,2653
1,2,2716
2,3,2691
3,4,2680
4,5,2647
5,6,2759
6,7,2743
7,8,2726
8,9,2712
9,10,2656


In [18]:
db

,UserID,SessionID,Timestamp,EventType,ProductID,Outcome
0,1,1,2024-07-07 18:00:26.959902,4,NaN,0
1,1,1,2024-03-05 22:01:00.072000,4,NaN,0
2,1,1,2024-03-23 22:08:10.568453,5,prod_8199,0
3,1,1,2024-03-12 00:32:05.495638,0,prod_4112,0
4,1,1,2024-02-25 22:43:01.318876,0,prod_3354,0
...,...,...,...,...,...,...
74812,1000,10,2024-05-11 22:48:45.500117,6,prod_1238,1
74813,1000,10,2024-03-29 04:09:32.514318,3,NaN,0
74814,1000,10,2024-02-09 02:58:56.128697,2,NaN,0
74815,1000,10,2024-04-30 16:19:48.002633,6,prod_2515,1


In [20]:
# Remove the 'prod_' prefix from the 'Product ID' column
db['ProductID'] = db['ProductID'].str.replace('prod_', '', regex=False)

db

,UserID,SessionID,Timestamp,EventType,ProductID,Outcome
0,1,1,2024-07-07 18:00:26.959902,4,NaN,0
1,1,1,2024-03-05 22:01:00.072000,4,NaN,0
2,1,1,2024-03-23 22:08:10.568453,5,8199,0
3,1,1,2024-03-12 00:32:05.495638,0,4112,0
4,1,1,2024-02-25 22:43:01.318876,0,3354,0
...,...,...,...,...,...,...
74812,1000,10,2024-05-11 22:48:45.500117,6,1238,1
74813,1000,10,2024-03-29 04:09:32.514318,3,NaN,0
74814,1000,10,2024-02-09 02:58:56.128697,2,NaN,0
74815,1000,10,2024-04-30 16:19:48.002633,6,2515,1


In [23]:
#df['ProductID'].fillna(df['ProductID'].max() + 1) replaces any NaN values in the ProductID column with the calculated value df['ProductID'].max() + 1.
db['ProductID'] = db['ProductID'].fillna(db['ProductID'].max() + 1)

db

,UserID,SessionID,Timestamp,EventType,ProductID,Outcome
0,1,1,2024-07-07 18:00:26.959902,4,10000.0,0
1,1,1,2024-03-05 22:01:00.072000,4,10000.0,0
2,1,1,2024-03-23 22:08:10.568453,5,8199.0,0
3,1,1,2024-03-12 00:32:05.495638,0,4112.0,0
4,1,1,2024-02-25 22:43:01.318876,0,3354.0,0
...,...,...,...,...,...,...
74812,1000,10,2024-05-11 22:48:45.500117,6,1238.0,1
74813,1000,10,2024-03-29 04:09:32.514318,3,10000.0,0
74814,1000,10,2024-02-09 02:58:56.128697,2,10000.0,0
74815,1000,10,2024-04-30 16:19:48.002633,6,2515.0,1


In [24]:
# Convert the column to numeric, forcing errors to NaN
#db['ProductID'] = pd.to_numeric(db['ProductID'], errors='coerce')

#db

In [25]:
# 3. Convert Categorical Data
# Encode categorical features using Label Encoding or One-Hot Encoding
# Here we use Label Encoding as an example
label_encoder = LabelEncoder()
db['ProductID'] = label_encoder.fit_transform(db['ProductID'])

In [26]:
db

,UserID,SessionID,Timestamp,EventType,ProductID,Outcome
0,1,1,2024-07-07 18:00:26.959902,4,8747,0
1,1,1,2024-03-05 22:01:00.072000,4,8747,0
2,1,1,2024-03-23 22:08:10.568453,5,7001,0
3,1,1,2024-03-12 00:32:05.495638,0,3029,0
4,1,1,2024-02-25 22:43:01.318876,0,2297,0
...,...,...,...,...,...,...
74812,1000,10,2024-05-11 22:48:45.500117,6,234,1
74813,1000,10,2024-03-29 04:09:32.514318,3,8747,0
74814,1000,10,2024-02-09 02:58:56.128697,2,8747,0
74815,1000,10,2024-04-30 16:19:48.002633,6,1478,1


In [27]:
# Drop the 'Amount' column
db = db.drop(columns=['Timestamp'])

In [29]:
db

,UserID,SessionID,EventType,ProductID,Outcome
0,1,1,4,8747,0
1,1,1,4,8747,0
2,1,1,5,7001,0
3,1,1,0,3029,0
4,1,1,0,2297,0
...,...,...,...,...,...
74812,1000,10,6,234,1
74813,1000,10,3,8747,0
74814,1000,10,2,8747,0
74815,1000,10,6,1478,1


In [30]:
# Define features and target
X = db.drop(['Outcome'], axis=1)
y = db['Outcome']

In [31]:
# Standardize data (optional)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [32]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)


In [40]:
# Train logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [44]:
# Predicting and evaluating the model
y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(f1_score(y_test, y_pred))

1.0
1.0
1.0
1.0


In [41]:
y_pred = model.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Classification Report:\n', classification_report(y_test, y_pred))
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))

Accuracy: 1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     19221
           1       1.00      1.00      1.00      3225

    accuracy                           1.00     22446
   macro avg       1.00      1.00      1.00     22446
weighted avg       1.00      1.00      1.00     22446

Confusion Matrix:
[[19221     0]
 [    0  3225]]


In [43]:
# Perform K-Fold Cross-Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=0)
cv_scores = cross_val_score(model, X, y, cv=kfold, scoring='r2')
cv_scores

array([1., 1., 1., 1., 1.])